### 선수별 1행 요약 (스텟 : 평균)

In [2]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [3]:
# data_type = "train"
# month = "07"
# category = "잔액정보"

# local
root_path = '../../data'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

#drive_folder = f'{root_path}/스탯티즈_Raw_data/'

In [4]:
train = pd.read_csv(f'{root_path}/KBO FA 투수 [2013-2024]_최종_전처리_ver03.csv', encoding='utf-8')

In [5]:
# 1. AAV 생성 및 결측치 제거
train["AAV"] = train["FA 계약 총액"] / train["FA 계약 연수"]
train = train.dropna(subset=["AAV"])

In [6]:
# 2. 필요 없는 컬럼 제거
drop_cols = [
    'FA 계약 총액', 'FA년도', '구단명', 'FA등급',
    '비고1', '구분', '잔류 여부', '구종1', '구종2', '구종1 구종가치'
]
train = train.drop(columns=drop_cols, errors='ignore')

In [7]:
# 3. 선수별 수치 피처 평균 요약
train_grouped = train.groupby('선수명').mean(numeric_only=True).reset_index()

In [8]:
# 4. 범주형 변수 원핫 인코딩 후 병합
cat_cols = ['투', '세부 포지션']
for col in cat_cols:
    if col in train.columns:
        temp = pd.get_dummies(train[[col, '선수명']], columns=[col])
        temp = temp.groupby('선수명').max().reset_index()
        train_grouped = pd.merge(train_grouped, temp, on='선수명', how='left')

In [9]:
# 6. 피처 / 타겟 정의
X = train_grouped.drop(columns=['AAV', '선수명'])
y = train_grouped['AAV']

In [10]:
# 7. train/val 분리
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
# 8. 모델 정의 및 성능 비교
models = {
    "RandomForest": RandomForestRegressor(random_state=42),
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, preds, squared=False)
    r2 = r2_score(y_val, preds)
    mes = np.mean(preds - y_val)
    
    print(f"📌 {name}")
    print(f"   RMSE: {rmse:,.0f} 원")
    print(f"   R²:   {r2:.4f}")
    print(f"   MES:   {mes:.4f}")
    print("-" * 30)

📌 RandomForest
   RMSE: 26,647 원
   R²:   0.7276
   MES:   8641.7857
------------------------------


### 튜닝

In [ ]:
# 1. 파라미터 탐색 범위 정의 (최소 범위로 간단하게)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [ ]:
# 2. GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    scoring='r2',
    cv=3,
    n_jobs=-1,
    verbose=1
)

In [ ]:
# 3. 학습
grid_search.fit(X_train, y_train)

In [ ]:
# 4. 최적 모델
best_model = grid_search.best_estimator_
print("📌 Best Params:", grid_search.best_params_)

In [ ]:
# 5. 검증 데이터 평가
val_preds = best_model.predict(X_val)
rmse = mean_squared_error(y_val, val_preds, squared=False)
r2 = r2_score(y_val, val_preds)

In [ ]:
print(f"✅ 튜닝 후 RMSE: {rmse:,.0f} 원")
print(f"✅ 튜닝 후 R²: {r2:.4f}")

- 별 의미 없다

In [ ]:
# train_grouped.to_csv("../../data/train_선수요약_AAV.csv", index=False, encoding="utf-8-sig")